In [1]:
from pymongo import MongoClient
from tqdm import tqdm
import numpy as np
import os
import pandas as pd
import pymongo
import xarray as xr

In [2]:
client = MongoClient()
db = client.eddies

In [3]:
def format(LON):
    if LON < 0:
        LON = LON + 360
    else:
        LON = LON
    return LON

In [4]:
rootdir = '/home/cz2397/data/cchdo-hyd/'

In [5]:
station = 0
overlap = 0
result = []

In [6]:
db.cchdo.create_index([('dat', pymongo.ASCENDING), ('loc', pymongo.ASCENDING)])

u'dat_1_loc_1'

In [7]:
for subdir, dirs, files in os.walk(rootdir):
    for file in tqdm(files):
        if file.endswith('hy1.nc'):
            path = os.path.join(subdir, file)
            DS = xr.open_dataset(path)
            DAT = DS.time.values[0]
            LAT = float(DS.latitude.values[0])
            LON = float(format(DS.longitude.values[0]))
            dat_var = np.timedelta64(4, 'D')
            dat_min = pd.to_datetime(DAT - dat_var)
            dat_max = pd.to_datetime(DAT + dat_var)
            ft = {
                'dat': {'$gt': dat_min, '$lt': dat_max},
                'loc': {'$within': {'$center': [[LON, LAT], 1]}}
            }
            count = db.cchdo.find(ft).count()
            station = station + 1
            if count != 0:
                overlap = overlap + 1
                result.append(path)

100%|██████████| 152/152 [00:06<00:00, 23.73it/s]


In [8]:
station

39790

In [9]:
overlap

6574

{'eddy_id':
 'cruise_id':
 'station_number':
 'station_lon':
 'station_lat':
}

In [10]:
result.sort()

In [11]:
result

['/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00007_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00008_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00009_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00017_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00018_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00019_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00020_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00022_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00023_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00024_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00027_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00045_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00046_00001_hy1.nc',
 '/home/cz2397/data/cchdo-hyd/101_a24_nc_hyd/a24_00047_00001_hy1.nc',
 '/home/cz2397/data/